In [ ]:
from google.colab import files
datafiles = files.upload()

Saving pro_stereotyped_type1.txt.dev.txt to pro_stereotyped_type1.txt.dev.txt
Saving pro_stereotyped_type1.txt.test.txt to pro_stereotyped_type1.txt.test.txt
Saving pro_stereotyped_type2.txt.dev.txt to pro_stereotyped_type2.txt.dev.txt
Saving pro_stereotyped_type2.txt.test.txt to pro_stereotyped_type2.txt.test.txt


In [ ]:
male_identifiers = ["him", "his", "he"]
female_identifiers = ["she", "her"]
gender_identifiers = male_identifiers + female_identifiers

def read_data(filename):
  file = open(filename, 'r')
  lines = file.readlines()
  sentences = []
  labels = []
  for line in lines:
    words = line.split(" ")
    sen = []
    for word in words:
      if word.isalpha() == False:
        word = ''.join(filter(str.isalpha, word))
      if word in gender_identifiers:
        if word in male_identifiers:
          label = 1 #male
        else:
          label = 0 #female
      elif word != '':
        sen.append(word.lower())
    sentences.append(sen)
    labels.append(label)
  return sentences, labels


In [ ]:
sentences1, labels1 = read_data("pro_stereotyped_type1.txt.dev.txt")
sentences2, labels2 = read_data("pro_stereotyped_type2.txt.dev.txt")
sentences3, labels3 = read_data("pro_stereotyped_type1.txt.dev.txt")
sentences4, labels4 = read_data("pro_stereotyped_type2.txt.dev.txt")

In [ ]:
trainX = sentences1 + sentences2
trainy = labels1 + labels2
testX = sentences3 + sentences4
testy = labels3 + labels4

In [ ]:
embeddingfiles = files.upload()

In [ ]:
def load_w2v_vectors(filename):
    with open(filename) as f:
        lines = [row.strip("\n") for row in f]
        num_words = len(lines) + 2 # unk and pad
        mat = torch.zeros(num_words, 50)
        vocab = {}
        for i, line in enumerate(lines):
            word, vec = line.split(" ", 1)
            vocab[word] = i + 2
            vec = [float(n) for n in vec.split()]
            mat[i + 2, :] = torch.tensor(vec)
    return mat, vocab

In [ ]:
wordvecmat, word2idx = load_w2v_vectors("enwiki_20180420_100d.txt")

In [ ]:
import torch

class LSTM(torch.nn.Module):
    def __init__(self, vocab_size, num_recurrent_neurons, bidirectional=False, embedding_size=50, pretrained_vectors=None):
        super().__init__()
        self.directions = 2 if bidirectional else 1
        self.emb = torch.nn.Embedding(vocab_size, embedding_size)
        if pretrained_vectors is not None:
            self.emb.data = pretrained_vectors

        self.rnn = torch.nn.LSTM(embedding_size, num_recurrent_neurons, bidirectional=bidirectional, batch_first=True)
        self.output = torch.nn.Linear(num_recurrent_neurons, 1)

    
    def forward(self, X):
        embeddings = self.emb(X)
        out, (h, c) = self.rnn(embeddings)

        if self.directions == 2:
            r = h.transpose(1, 0).contiguous().view(h.shape[1], -1)
        else:
            r = h.squeeze()

        r = self.dropout(r)
        out = self.output(r)
        return torch.sigmoid(out)


In [ ]:
def vocab_transform(sents, word2idx, unk_idx):
    return torch.nn.utils.rnn.pad_sequence(
        [torch.tensor([word2idx.get(w, unk_idx) for w in tokenize(sent.lower())]) 
        for sent in sents], batch_first=True)

In [ ]:
batch_size = 50
batches = [(trainX[i:i+batch_size], trainy[i:i+batch_size]) for i in range(0, len(trainX), batch_size)]
clf = LSTM(i, 256, embedding_size=100, pretrained_vectors=wordvecmat)
criterions = torch.nn.BCELoss()
optimizers = torch.optim.Adam(clf.parameters(), lr=0.00001)

for epoch in range(1):
    clf.train()
    total_loss = 0
    for x, y in batches:
        optimizers.zero_grad()
        encoded = vocab_transform(x)
        ypred = clf(encoded).squeeze(0)
        losses = criterions(ypred, y)
        total_loss += losses 
        losses.backward()
        optimizers.step()
    print(f"epoch {epoch}")
    print(total_loss / (len(trainX)))


In [ ]:
from sklearn.metrics import classification_report
batches = [(testX[i:i+batch_size], testy[i:i+batch_size]) for i in range(0, len(testX), batch_size)]
preds = []

clf.eval()
for x, y in batches:
    encoded = vocab_transform(x)
    ypred = [y.item() for y in clf(encoded).squeeze().round()]
    preds.extend(ypred)
print(classification_report(testy, ypred))